# Тренирање на Tacotron 2

Во оваа вежба ќе ги претставиме чекорите потребни за тренирање на еден од најпопуларните модели за креирање на текст-во-говор.  

Ги содржи следните секции:

  1. Tacotron2 и NeMo - Вовед во моделот Tacotron2.

  2. LJSpeech - Како да се истренира Tacotron2 на податочното множество LJSpeech.
  3. Различни јазици - Како да се соберат аудио податоци за тренирање на Tacotron2 за различни гласови и јазици.

# Лиценца

> Copyright 2020 NVIDIA. All Rights Reserved.
> 
> Apache лиценца, верзија 2.0. Оваа датотека мора да се користи согласно лиценцата која може да се преземе на следниот линк:
> 
>     http://www.apache.org/licenses/LICENSE-2.0
> 
> Ако не е поинаку специфицирано, кодот е дистрибуиран на база "AS IS", без услови и гаранции. Видете ја лиценцата за подетални информации во однос на ограничувањата.

In [ ]:
"""
Оваа скрипта може да се извршува локално, доколку имате на располагање графичка картичка GPU или на Google Colab.
Инструкции за подесување на Colab:
1. Отворете нова Python 3 notebook.
2. Импортирајте од GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Поврзете се со GPU инстанца (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Извршете ја оваа ќелија за да ги вклучите сите зависности = dependencies# .
"""
# # Ако го користите Colab извршете ја оваа ќелија.
# !apt-get install sox libsndfile1 ffmpeg
# !pip install wget unidecode
# BRANCH = 'r1.0.0rc1'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[tts]

# Tacotron2 и NeMo

Tacotron2 е невронска мрежа која конвертира текст карактери во мел спектрограм. Повеќе детали може да се прочитаат на страницата на Nvidia's [Tacotron2 Model Card](https://ngc.nvidia.com/catalog/models/nvidia:nemo:tts_en_tacotron2), или од оригиналниот труд (https://arxiv.org/abs/1712.05884). Во оваа скрипта ќе се задржиме на практична имплементација.

Tacotron2 како и повеќето NeMo модели е дефиниран како LightningModule, дозволувајќи лесно тренирање со PyTorch Lightning, и параметризирање преку конфигурација дефинирана во yaml датотека.

Прво ќе видиме како да вчитаме NeMo претрениран модел и истиот да го искористиме за генерирање спектрограми.

In [ ]:
# Вчитување на Tacotron2Model
from nemo.collections.tts.models import Tacotron2Model
from nemo.collections.tts.models.base import SpectrogramGenerator

# Да ги видиме претренираните модели
print(Tacotron2Model.list_available_models())

In [ ]:
# Можеме да вчитаме претрениран модел на следниот начин
model = Tacotron2Model.from_pretrained("tts_en_tacotron2")

In [ ]:
# Tacotron2 е SpectrogramGenerator
assert isinstance(model, SpectrogramGenerator)

# SpectrogramGenerators во NeMo има 2 помошни функции:
#   1. parse(str_input: str, **kwargs) зема стринг на англиски јазик и продуцира тензор со токени
#   2. generate_spectrogram(tokens: 'torch.tensor', **kwargs) го зема тензорот со токени и генерира спектрограм
# Да ги пробаме:
tokens = model.parse(str_input = "Hey, this produces speech!")
spectrogram = model.generate_spectrogram(tokens = tokens)

# Сега можеме да ги визуелизираме генерираните спектрограми
# Ако сакаме да генерираме говор, мора да користиме vocoder заедно со генератор на spectrogram.
from matplotlib.pyplot import imshow
from matplotlib import pyplot as plt
%matplotlib inline
imshow(spectrogram.cpu().detach().numpy()[0,...], origin="lower")
plt.show()

# Тренирање

Да видиме како може да се претренира Tacotron 2 одново.



In [ ]:
# NeMo скриптите за тренирање се наоѓаат во фолдерот examples/. 
# Потребно е да ги преземеме скриптите tacotron2.py и tacotron2.yaml file
!wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.0.0rc1/examples/tts/tacotron2.py
!mkdir conf && cd conf && wget https://raw.githubusercontent.com/NVIDIA/NeMo/r1.0.0rc1/examples/tts/conf/tacotron2.yaml && cd ..

Да ја погледнеме датотеката tacotron2.py

```python
import pytorch_lightning as pl

from nemo.collections.common.callbacks import LogEpochTimeCallback
from nemo.collections.tts.models import Tacotron2Model
from nemo.core.config import hydra_runner
from nemo.utils.exp_manager import exp_manager


# hydra_runner е NeMo врапер
# ја бара конфигурациската датотека tacotron2.yaml во conf фолдерот
# Hydra ја парсира yaml датотеката и ја враќа како Omegaconf DictConfig
@hydra_runner(config_path="conf", config_name="tacotron2")
def main(cfg):
    # Дефинираме Lightning trainer
    trainer = pl.Trainer(**cfg.trainer)
    # exp_manager е NeMo конструкција која помага при вчитување и зачувување на чекпоинтс
    exp_manager(trainer, cfg.get("exp_manager", None))
    # Го дефинира Tacotron 2 моделот, тренингот и валидацијата
    model = Tacotron2Model(cfg=cfg.model, trainer=trainer)
    # Додава callbacks
    lr_logger = pl.callbacks.LearningRateMonitor()
    epoch_time_logger = LogEpochTimeCallback()
    trainer.callbacks.extend([lr_logger, epoch_time_logger])
    # Ја повикува lightning trainer's fit() за тренирање на моделот
    trainer.fit(model)


if __name__ == '__main__':
    main()  # noqa pylint: disable=no-value-for-parameter
```

Да ја погледнеме yaml конфигурацијата

```yaml
name: &name Tacotron2
sample_rate: &sr 22050
# <PAD>, <BOS>, <EOS> ќе бидат додадени од tacotron2.py скриптата
labels: &labels [' ', '!', '"', "'", '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H',
                 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']',
                 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't',
                 'u', 'v', 'w', 'x', 'y', 'z']
n_fft: &n_fft 1024
n_mels: &n_mels 80
fmax: &fmax null
n_stride: &n_window_stride 256
pad_value: &pad_value -11.52
train_dataset: ???
validation_datasets: ???
```

Првиот дел од yaml дефинира некои параметри користени од Tacotron. Може да забелешите дека ратата на семплирање е 22050 за податочното множество LJSpeech.

Можеме да забележиме и дека `train_dataset: ???` и `validation_datasets: ???`. ??? укажува дека овие вредности мора да бидат пренесени преку командна линија, инаку скриптата нема да работи.

Понатаму ги гледаме pytorch lightning trainer параметрите.

```yaml
trainer:
  gpus: 1 # број на gpus
  max_epochs: ???
  num_nodes: 1
  accelerator: ddp
  accumulate_grad_batches: 1
  checkpoint_callback: False  # од страна на exp_manager
  logger: False  # од страна на exp_manager
  gradient_clip_val: 1.0
  flush_logs_every_n_steps: 1000
  log_every_n_steps: 200
  check_val_every_n_epoch: 25
```

Овие вредности може да се сменат или преку уредување на yaml или преку командна линија.

Да преземеме аудио податоци и да го тестираме Tacotron2.

In [ ]:
!wget https://github.com/NVIDIA/NeMo/releases/download/v0.11.0/test_data.tar.gz && mkdir -p tests/data && tar xzf test_data.tar.gz -C tests/data

# Tacotron2 бара .json датотеки за дефинирање на тренинг и валидациските податоци.
!cat tests/data/asr/an4_val.json

# Забелешка: Земеното податочно множество не е доволно за вистинско тренирање на Tacotron 2. 
# Нема да резултира во квалитетен Tacotron 2 модел, но е доволно за илустрирање како да тренирате модел.
# За квалитетен Tacotron 2 потребна е голема порција податоци и тренирањето трае неколку недели.
!python tacotron2.py sample_rate=16000 train_dataset=tests/data/asr/an4_train.json validation_datasets=tests/data/asr/an4_val.json trainer.max_epochs=3 trainer.accelerator=null trainer.check_val_every_n_epoch=1

# Податоци за тренирање

За тренирање на Tacotron2, препорачливо е да употребите податочно множество со следните параметри:
  - Рата на семплирање од 22050Hz или повисока
  - Еден говорник
  - Говорот треба да содржи различни фонеми
  - Датотеките треба да се со должина од 1-10 секунди
  - Датотеките не треба да имаат „молк“ на почетокот и на крајот
  - Датотеките не треба да содржат долги сегменти со „молк“

Откако ќе се добијат податоците и ќе се поделат на тренирачки, валидациски и тестирачки податоци, треба да се конструираат .json датотеки кои ќе му кажат на NeMo каде се наоѓаат овие аудио датотеки.

На пример:

```json
{"audio_filepath": "/path/to/audio1.wav", "text": "the transcription", "duration": 0.82}
{"audio_filepath": "/path/to/audio2.wav", "text": "the other transcription", "duration": 2.1}
...
```
Времетраењето е во секунди.

Последно, обновете ги лабелите во Tacotron 2 yaml конфигурацијата доколку вашите податоци содржат различни карактери.

Сега сте подготвени да ја стартувате скриптата:
```bash
python tacotron2.py train_dataset=YOUR_TRAIN.json validation_datasets=YOUR_VAL.json trainer.gpus=-1
```

Честитки! Научивте како да направите модел за транслација на текст во говор!